<a href="https://colab.research.google.com/github/CMU-313/spring23-nodebb-team-dj-kew/blob/K%2Fjupyter-notebook-setup/P4_Jupyter_Notebook_Setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure to copy the following files from https://github.com/CMU-313/NodeBB/pull/186/files

- `career-model/requirements.txt`
- `career-model/model.pkl`

In [ ]:
!python --version
!pip install -r requirements.txt

Python 3.9.16
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Load the model and test dataset

In [ ]:
import pandas as pd
import joblib
from pydantic import BaseModel, Field
from pydantic.tools import parse_obj_as

# Pydantic Models
class Student(BaseModel):
    student_id: str = Field(alias="Student ID")
    gender: str = Field(alias="Gender")
    age: str = Field(alias="Age")
    major: str = Field(alias="Major")
    gpa: str = Field(alias="GPA")
    extra_curricular: str = Field(alias="Extra Curricular")
    num_programming_languages: str = Field(alias="Num Programming Languages")
    num_past_internships: str = Field(alias="Num Past Internships")

    class Config:
        allow_population_by_field_name = True

class PredictionResult(BaseModel):
    good_employee: int


# Main Functionality
def predict(student):
    '''
    Returns a prediction on whether the student will be a good employee
    based on given parameters by using the ML model
    Parameters
    ----------
    student : dict
        A dictionary that contains all fields in Student
    
    Returns
    -------
    dict
        A dictionary satisfying type PredictionResult, contains a single field
        'good_employee' which is either 1 (will be a good employee) or 0 (will
        not be a good employee)
    '''
    # Use Pydantic to validate model fields exist
    student = parse_obj_as(Student, student)

    clf = joblib.load('./model.pkl')

    student = student.dict(by_alias=True)
    query = pd.DataFrame(student, index=[0])
    prediction = clf.predict(query) # TODO: Error handling ??

    return { 'good_employee': prediction[0] }

# Sample run
student = {
    "student_id": "student1",
    "major": "Computer Science",
    "age": "20",
    "gender": "M",
    "gpa": "4.0",
    "extra_curricular": "Men's Basketball",
    "num_programming_languages": "0",
    "num_past_internships": "0"
}
predict(student)

{'good_employee': 1}

Plot the distribution of the test dataset across all features (except Student ID) using any visualization library of your choice (e.g. pandas, matplotlib, seaborn, plotly, etc.). You should choose the appropriate visualization for each feature.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('student_data.csv')
df_good = df.loc[(df['Good Candidate'] == 1)]
df_bad = df.loc[(df['Good Candidate'] == 0)]

# Distribution of GPA, based on Good Candidate
df.plot.hist(column=["GPA"], by="Good Candidate", figsize=(10, 8))

# Distribution of every other field
fields = ['Gender', 'Age', 'Major', 'Extra Curricular', 'Num Programming Languages', 'Num Past Internships']
for field in fields:
  fig, ax = plt.subplots()
  df[field].value_counts().sort_index().plot(ax=ax, kind='bar', title=field)

Predict the output of the test dataset using the model

In [ ]:
student = {
    "student_id": "student1",
    "major": "Computer Science",
    "age": "20",
    "gender": "M",
    "gpa": "4.0",
    "extra_curricular": "Men's Basketball",
    "num_programming_languages": "0",
    "num_past_internships": "0"
}
predict(student)

# Keep track of outputs, predicted vs actual for accuracy metrics later
predicted_outputs = []
actual_outputs = []

# Iterate over dataset and predict each student (THIS TAKES ABOUT 3 MINUTES)
for index, row in df.iterrows():
  fields = ["student_id", "major", "age", "gender", "gpa", "extra_curricular", "num_programming_languages", "num_past_internships"]
  col_names = ['Student ID', 'Major', 'Age', 'Gender', 'GPA', 'Extra Curricular', 'Num Programming Languages', 'Num Past Internships']

  # Construct a student object to be predicted
  student = dict()
  for i in range(len(fields)):
    curr_field = fields[i]
    curr_col = col_names[i]
    student[curr_field] = row[curr_col]

  # Get the outputs, predicted and actual
  predicted_outputs.append(predict(student)['good_employee'])
  actual_outputs.append(row['Good Candidate'])

Report the accuracy of the model, and the confusion matrix

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

print("Accuracy score: ", accuracy_score(actual_outputs, predicted_outputs))
print("Confusion matrix: \n", confusion_matrix(actual_outputs, predicted_outputs))

Accuracy score:  0.832
Confusion matrix: 
 [[221  42]
 [ 42 195]]
